In [158]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import linear_model

In [159]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [208]:
# import data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# combine common features for processing together
combined = train.drop('SalePrice', axis = 1).append(test)

In [161]:
# check for percentage of missing values
combined.isnull().sum() * 100 / combined.shape[0]

Id                0.000000
MSSubClass        0.000000
MSZoning          0.137033
LotFrontage      16.649538
LotArea           0.000000
Street            0.000000
Alley            93.216855
LotShape          0.000000
LandContour       0.000000
Utilities         0.068517
LotConfig         0.000000
LandSlope         0.000000
Neighborhood      0.000000
Condition1        0.000000
Condition2        0.000000
BldgType          0.000000
HouseStyle        0.000000
OverallQual       0.000000
OverallCond       0.000000
YearBuilt         0.000000
YearRemodAdd      0.000000
RoofStyle         0.000000
RoofMatl          0.000000
Exterior1st       0.034258
Exterior2nd       0.034258
MasVnrType        0.822199
MasVnrArea        0.787941
ExterQual         0.000000
ExterCond         0.000000
Foundation        0.000000
BsmtQual          2.774923
BsmtCond          2.809181
BsmtExposure      2.809181
BsmtFinType1      2.706406
BsmtFinSF1        0.034258
BsmtFinType2      2.740665
BsmtFinSF2        0.034258
B

In [162]:
lessMissing = combined.isnull().sum() * 100 / combined.shape[0] < 15

In [163]:
combined = combined.loc[:, lessMissing]

In [164]:
combined.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           4
LotArea            0
Street             0
LotShape           0
LandContour        0
Utilities          2
LotConfig          0
LandSlope          0
Neighborhood       0
Condition1         0
Condition2         0
BldgType           0
HouseStyle         0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
RoofStyle          0
RoofMatl           0
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
ExterQual          0
ExterCond          0
Foundation         0
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Heating            0
HeatingQC          0
CentralAir         0
Electrical         1
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       2
BsmtHalfBath       2
FullBath     

In [165]:
combined['GarageCars'].fillna(combined['GarageCars'].mode().values[0], inplace = True)
combined['MSZoning'].fillna(combined['MSZoning'].mode().values[0], inplace = True)
combined['Utilities'].fillna(combined['Utilities'].mode().values[0], inplace = True)

In [166]:
combined.groupby(['OverallQual', 'KitchenQual']).size()

OverallQual  KitchenQual
1            Fa               3
             TA               1
2            Fa               3
             Gd               1
             TA               9
3            Fa               7
             Gd               3
             TA              30
4            Fa              23
             Gd              21
             TA             182
5            Ex              15
             Fa              21
             Gd             126
             TA             662
6            Ex               7
             Fa              12
             Gd             244
             TA             468
7            Ex              15
             Fa               1
             Gd             455
             TA             129
8            Ex              52
             Gd             279
             TA              11
9            Ex              88
             Gd              19
10           Ex              28
             Gd               3
dtype: int64

In [167]:
significant = ['OverallQual', 'OverallCond', 'MSSubClass', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'BsmtHalfBath', 'FullBath', 'KitchenAbvGr', 'GarageArea', 'EnclosedPorch']
combined = combined.loc[:, significant]
combined.isnull().sum()


OverallQual      0
OverallCond      0
MSSubClass       0
BsmtFinSF2       1
TotalBsmtSF      1
1stFlrSF         0
GrLivArea        0
BsmtHalfBath     2
FullBath         0
KitchenAbvGr     0
GarageArea       1
EnclosedPorch    0
dtype: int64

In [168]:
combined['BsmtFinSF2'].fillna(combined['BsmtFinSF2'].mean(), inplace = True)
combined['TotalBsmtSF'].fillna(combined['TotalBsmtSF'].mean(), inplace = True)
combined['BsmtHalfBath'].fillna(combined['BsmtHalfBath'].mode().values[0], inplace = True)
combined['GarageArea'].fillna(np.floor(combined['GarageArea'].mean()), inplace = True)

In [169]:
combined.dtypes

OverallQual        int64
OverallCond        int64
MSSubClass         int64
BsmtFinSF2       float64
TotalBsmtSF      float64
1stFlrSF           int64
GrLivArea          int64
BsmtHalfBath     float64
FullBath           int64
KitchenAbvGr       int64
GarageArea       float64
EnclosedPorch      int64
dtype: object

In [170]:
# convert some columns to categorical
categorical = ['BsmtHalfBath', 'OverallQual', 'OverallCond', 'MSSubClass', 'FullBath', 'KitchenAbvGr']
for column in categorical:
    combined[column] = combined[column].astype('category')

In [171]:
# normalise
minMaxScaler = preprocessing.MinMaxScaler()
combined.loc[:, ['BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea', 'EnclosedPorch']] = minMaxScaler.fit_transform(combined.loc[:, ['BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea', 'EnclosedPorch']])

In [172]:
# label encoding for categorical columns
le = preprocessing.LabelEncoder()
combined.loc[:, categorical] = combined.loc[:, categorical].apply(lambda x: le.fit_transform(x))

In [173]:
combined.head(10)

,OverallQual,OverallCond,MSSubClass,BsmtFinSF2,TotalBsmtSF,1stFlrSF,GrLivArea,BsmtHalfBath,FullBath,KitchenAbvGr,GarageArea,EnclosedPorch
0,6,4,5,0.00000,0.140098,0.109641,0.259231,0,2,1,0.368280,0.000000
1,5,7,0,0.00000,0.206547,0.194917,0.174830,1,2,1,0.309140,0.000000
2,6,4,5,0.00000,0.150573,0.123083,0.273549,0,2,1,0.408602,0.000000
3,6,4,6,0.00000,0.123732,0.131695,0.260550,0,1,1,0.431452,0.268775
4,7,4,5,0.00000,0.187398,0.170342,0.351168,0,2,1,0.561828,0.000000
5,4,4,4,0.00000,0.130278,0.097038,0.193670,0,1,1,0.322581,0.000000
6,7,4,0,0.00000,0.275941,0.285654,0.256217,0,2,1,0.427419,0.000000
7,6,5,5,0.02097,0.181178,0.162361,0.330821,0,2,1,0.325269,0.225296
8,6,4,4,0.00000,0.155810,0.144507,0.271289,0,2,2,0.314516,0.202569
9,4,5,15,0.00000,0.162193,0.156060,0.139977,0,1,2,0.137769,0.000000


In [174]:
# split combined into train and test again
train['SalePrice'] = train['SalePrice'].astype('float')
train = pd.merge(combined.iloc[0:train.shape[0], :], train.loc[:, ['SalePrice']], on = train.index).drop('key_0', axis = 1)
test = combined.iloc[train.shape[0]:combined.shape[0], :]

In [192]:
# fit linear regression
lm = linear_model.LinearRegression()
lm.fit(train.iloc[:, 0:train.shape[1]-1], train['SalePrice'])
predictions = lm.predict(test)

In [210]:
submission = pd.DataFrame({'Id': test.Id.values, 'SalePrice': predictions})
submission.to_csv('submission.csv', index = False)

In [209]:
test.Id

0       1461
1       1462
2       1463
3       1464
4       1465
5       1466
6       1467
7       1468
8       1469
9       1470
10      1471
11      1472
12      1473
13      1474
14      1475
15      1476
16      1477
17      1478
18      1479
19      1480
20      1481
21      1482
22      1483
23      1484
24      1485
25      1486
26      1487
27      1488
28      1489
29      1490
30      1491
31      1492
32      1493
33      1494
34      1495
35      1496
36      1497
37      1498
38      1499
39      1500
40      1501
41      1502
42      1503
43      1504
44      1505
45      1506
46      1507
47      1508
48      1509
49      1510
50      1511
51      1512
52      1513
53      1514
54      1515
55      1516
56      1517
57      1518
58      1519
59      1520
60      1521
61      1522
62      1523
63      1524
64      1525
65      1526
66      1527
67      1528
68      1529
69      1530
70      1531
71      1532
72      1533
73      1534
74      1535
75      1536
76      1537